In [1]:
# add notebook directory to path
# get the notebook directory
import os
import sys
from pathlib import Path

from sklearn.preprocessing import StandardScaler

# add the notebook directory to the path
notebook_dir = Path(os.getcwd()).absolute().parent
sys.path.append(str(notebook_dir / 'src' / 'residPCA'))

from residualize.linear import linear_regression, LinearRegressionResiduals

In [2]:
import numpy as np

beta = np.random.randn(5000, 5)
C = np.random.random(size=(10000, 5))

# get a matrix C that is like a one-hot encoding
# C = np.random.randint(0, 5, (10000, 1))
# C = np.eye(5)[C.flatten()]

X = C @ beta.T + 0.1 * np.random.randn(10000, 5000)

In [3]:
# C = StandardScaler().fit_transform(C)

In [4]:
%%timeit
res = linear_regression(X, C, method='lstsq')

1.08 s ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
res = LinearRegressionResiduals(method="lstsq").fit_transform(C, X)

923 ms ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
res = linear_regression(X, C, method='lstsq', standard_scale_data=True, standard_scale_residuals=False)
resid = LinearRegressionResiduals(method="lstsq", standard_scale_covars=False, include_intercept=True, standard_scale_residuals=False)
res_fittrans = resid.fit_transform(C, X)
np.allclose(res, res_fittrans)

False

In [4]:
resid

LinearRegressionResiduals()

In [10]:
resid.named_steps["scaler"].mean_

array([-0.08627748,  0.44049064, -0.48084707, ..., -0.73098672,
        0.77346593,  0.45467575])

In [9]:
res_fittrans

array([[ 0.53787191, -0.57488127, -1.58173891, ...,  0.50823757,
         0.0114501 , -0.35815696],
       [ 0.53235115,  0.96855667, -0.34617454, ...,  0.19928449,
        -0.33332286,  0.31640997],
       [-2.04249257,  0.18039241,  0.13013048, ..., -1.45837972,
         0.93891579,  0.60618759],
       ...,
       [ 1.17009593,  0.55757076, -1.47182204, ..., -0.18201581,
        -1.42735911,  0.37178153],
       [ 0.42036454, -0.13243313,  0.50631383, ..., -0.78260858,
         1.06323979,  1.15173399],
       [-1.23568927,  0.58644044, -0.72916739, ..., -0.83040571,
        -0.77865512, -0.32239118]])

In [10]:
res

array([[ 0.63475635,  0.18433423, -1.39564422, ..., -0.24091992,
         0.01320406, -0.51032163],
       [ 1.10515743,  1.25127505, -0.95165265, ..., -0.94194348,
        -0.32846619, -0.58835418],
       [-0.65196486,  1.01856955, -0.60458984, ..., -1.31578772,
         0.94373595, -0.40225183],
       ...,
       [ 0.79891396,  0.16405881, -1.06448339, ..., -0.04412506,
        -1.42733006,  0.23213415],
       [-0.26068053, -0.75341423,  0.88246208, ...,  0.54876164,
         1.05965285,  1.30573597],
       [-1.93303041, -1.25681069,  0.5048934 , ...,  1.26175087,
        -0.78577869,  0.95459368]])

In [11]:
np.allclose(beta, resid.get_effectsizes().T[:, :-1])

False

In [12]:
resid.get_effectsizes().T

array([[ 0.14701382,  0.01982158, -2.78405969,  1.14510611,  1.13435404,
         0.16079588],
       [-1.16615898, -1.16128674,  0.29641028,  0.98865046,  2.99201307,
        -0.77450648],
       [-1.25155958,  0.64188721, -2.27611995,  1.05397433,  1.03928464,
         1.992216  ],
       ...,
       [-0.39265268, -0.46063817, -2.52137839,  1.90964503,  0.91637219,
         0.9060595 ],
       [ 2.23705913,  0.49332148, -3.1814519 , -0.66411031, -0.14068539,
        -0.99265341],
       [ 1.44962807, -2.66769217, -0.20737186,  1.14899391,  0.60731541,
        -1.74432098]])

In [11]:
beta

array([[-1.60028841, -0.10095152,  0.89530538,  0.28697016,  0.47714505],
       [-0.70931562,  0.83104534, -0.40399462,  1.11692362, -0.02065567],
       [ 0.79558628, -0.06199   , -0.13955363,  0.94251921, -1.42788958],
       ...,
       [-1.19768626,  0.680563  ,  0.69213821,  0.19101885,  0.67968774],
       [ 0.91875669,  0.15416259,  0.63255619,  1.80984947,  1.106385  ],
       [-0.2784853 ,  0.02430724, -0.87236505,  1.23977626,  2.33557398]])

In [15]:
sc = StandardScaler().fit_transform(C)

In [16]:
sc

array([[ 5.90146462e-01,  1.47792611e+00,  1.40958522e+00,
        -3.08063361e+01, -9.43549749e-01],
       [-4.93268254e+00,  1.66901694e+00, -3.83662461e+01,
         1.56821754e+00,  7.22430057e-01],
       [-4.79033242e+00,  1.16551014e+00,  1.60286227e+00,
        -7.03912630e+00, -8.21278518e+01],
       ...,
       [ 1.04926866e+00, -8.33436341e-01, -9.83370436e-01,
        -1.91412637e+00,  2.83694175e-01],
       [ 4.20014956e-01, -1.68449879e-01, -1.29886794e+02,
        -1.80965746e+02,  1.14697408e+00],
       [-1.44268090e+00, -4.87638714e+00, -4.74087980e+01,
         9.71985447e-01,  1.69426982e+00]])

In [5]:
StandardScaler().fit_transform(C)

array([[-0.49812429, -0.49311521, -0.49984375,  1.9950137 , -0.50764499],
       [ 2.00753109, -0.49311521, -0.49984375, -0.50124969, -0.50764499],
       [-0.49812429, -0.49311521, -0.49984375, -0.50124969,  1.96988057],
       ...,
       [ 2.00753109, -0.49311521, -0.49984375, -0.50124969, -0.50764499],
       [-0.49812429, -0.49311521, -0.49984375, -0.50124969,  1.96988057],
       [-0.49812429,  2.02792368, -0.49984375, -0.50124969, -0.50764499]])

In [10]:
res_fittrans

array([[ 0.86857462, -0.9773196 , -1.76294478, ...,  0.64231043,
         0.31406272,  1.00260493],
       [-2.0476132 , -1.60264333,  1.26727146, ...,  1.08917276,
        -0.37793303,  0.90646029],
       [-0.57893127, -0.26291433, -0.95746459, ...,  0.69206071,
         0.64360386,  0.924078  ],
       ...,
       [-1.70627209, -0.61136788, -0.11264166, ..., -0.17215184,
         0.05098688, -1.03303906],
       [-0.87867462, -1.13130186,  0.1320789 , ..., -0.2679621 ,
         0.03493907, -0.34853973],
       [-0.55553995,  1.13399028, -1.91319126, ...,  1.82802736,
        -1.30984347,  1.18814588]])

In [1]:
resid

NameError: name 'resid' is not defined

In [7]:
resid.get_effectsizes()

array([[-0.01946134,  0.03344876,  0.00911665, ..., -0.0050101 ,
         0.05933548,  0.01624387],
       [ 0.00215077, -0.01549115, -0.00115449, ..., -0.00144169,
         0.00229403, -0.01006969],
       [ 0.03952053, -0.00938263, -0.02650301, ...,  0.00543544,
        -0.02281438, -0.01109381],
       [ 0.00918529,  0.00822027, -0.00046944, ..., -0.00655149,
        -0.02084236,  0.03505526],
       [-0.03002383, -0.01576183,  0.01814639, ...,  0.00727426,
        -0.01702115, -0.02873751]])

In [6]:
res_fittrans

array([[ 0.76176659,  1.64247795, -0.20412727, ...,  0.04713881,
         0.29279819, -1.19268851],
       [ 0.33012411,  0.88777641,  0.77074149, ...,  0.09490215,
         0.26341367, -0.06789525],
       [-0.47260226, -1.44569502,  0.65083525, ...,  0.23198664,
        -0.08250079,  0.91159548],
       ...,
       [ 0.91653242, -0.38898683,  0.57879831, ..., -1.1426551 ,
        -1.03164131, -0.00605048],
       [-1.2248389 ,  0.05934534,  1.59739126, ...,  0.81814278,
         1.68794313, -1.0363815 ],
       [ 0.41914123,  1.17996103,  0.84849702, ..., -1.4214678 ,
         1.59525818,  0.23453582]])

In [7]:
res

array([[ 0.76183561,  1.64274016, -0.20416747, ...,  0.04715019,
         0.29288306, -1.19271723],
       [ 0.33015402,  0.88791814,  0.77089326, ...,  0.09492505,
         0.26349002, -0.06789688],
       [-0.47264508, -1.44592581,  0.65096341, ...,  0.23204263,
        -0.0825247 ,  0.91161743],
       ...,
       [ 0.91661546, -0.38904893,  0.57891228, ..., -1.14293085,
        -1.03194033, -0.00605062],
       [-1.22494988,  0.05935481,  1.5977058 , ...,  0.81834022,
         1.68843239, -1.03640645],
       [ 0.41917921,  1.1801494 ,  0.8486641 , ..., -1.42181083,
         1.59572057,  0.23454146]])

In [15]:
scExp = residPCA(count_matrix_path="/Users/shayecarver/residPCA/examples/example_data.h5ad", 
               vars_to_regress=['Batch','celltype','total_counts', 'pct_counts_mt','Age','Sex'],
               object_columns=['celltype','Sex','Batch'], 
               save_image_outputs=False, 
               BIC=True, 
               vargenes_Stand_resid=3000,
               n_PCs = 150,
               random_seed=7, 
               vargenes_IterPCA=3000, 
               #path_to_directory = "./", 
               #basename=f'Morab_RNA_3kvargenes_seurat',
               variable_genes_flavor="seurat",  
               global_ct_cutoff=0.2)  
scExp.Normalize() 
scExp.Standardize()
scExp.StandardPCA_fit()
scExp.residPCA_fit()
scExp.Iter_PCA_fit()
# identify global versus cell type specific states
scExp.ID_Global_CellType_States()


Finding most variable genes using 'seurat' flavor.
Using "seurat" flavor to compute variable genes in Iterative PCA on ASC.


/Users/shayecarver/opt/anaconda3/envs/condPCA/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:279: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(


Using "seurat" flavor to compute variable genes in Iterative PCA on EX.


/Users/shayecarver/opt/anaconda3/envs/condPCA/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:279: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(


Using "seurat" flavor to compute variable genes in Iterative PCA on INH.


...

/Users/shayecarver/opt/anaconda3/envs/condPCA/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:279: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(


Using "seurat" flavor to compute variable genes in Iterative PCA on OPC.


/Users/shayecarver/opt/anaconda3/envs/condPCA/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:279: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(


Reducing the number of PCs for PER.END Iterative PCA to 56.
Using "seurat" flavor to compute variable genes in Iterative PCA on PER.END.


/Users/shayecarver/opt/anaconda3/envs/condPCA/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:279: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(
